In [ ]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

# Load the text file containing the sentences.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
lines = []
with open('/content/drive/MyDrive/sentiment labelled sentences/imdb_labelled.txt') as theFile1:
  for y in theFile1:
        lines.append(y.rstrip('\n'))
print(len(lines))
with open('/content/drive/MyDrive/sentiment labelled sentences/amazon_cells_labelled.txt') as theFile2:
  for y in theFile2:
        lines.append(y.rstrip('\n'))
print(len(lines))
with open('/content/drive/MyDrive/sentiment labelled sentences/yelp_labelled.txt') as theFile3:
    for y in theFile3:
        lines.append(y.rstrip('\n'))
print(len(lines))
print(lines[0])

# Create empty lists to store sentences and labels
sentences = []
labels = []

# Separate sentences and labels
sentences, labels = zip(*[line.split('\t') for line in lines])

print(sentences[0])
print(labels[0])


# Write a script to collect all the words and put them on a list.

In [ ]:

allWords=[]
for x in sentences:
    wx=x.split(' ')
    for i in wx:
        allWords.append(i)

print(allWords)
print(len(allWords))

# Collect the unique words.

In [ ]:
uniqueWords=set(allWords)

print(uniqueWords)
vocabSize=len(uniqueWords)
print(vocabSize)

uW=list(uniqueWords)
print(uW)
print(uW[:5])

sorteduW = sorted(uW)
print(sorteduW)

# Tokenize Sentences and Prepare Data for Model Training.

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Create DataFrame
data = {'Words': list(uniqueWords)}
word_df = pd.DataFrame(data)

# Use LabelEncoder to perform integer encoding on words
word_label_encoder = LabelEncoder()
word_df['Word_encoded'] = word_label_encoder.fit_transform(word_df['Words'])

# Display the DataFrame for encoded words
print(word_df)

# Create a DataFrame for sentences
sentence_df = pd.DataFrame({'Sentences': sentences})

# Tokenize the sentences into words
sentence_df['Tokenized_Sentences'] = sentence_df['Sentences'].apply(lambda sentence: sentence.split())

# Use the encoded words to encode the sentences
sentence_df['Sentences_encoded'] = sentence_df['Tokenized_Sentences'].apply(
    lambda tokens: [word_label_encoder.transform([word])[0] for word in tokens]
)

# Display the final DataFrame
print(sentence_df['Sentences_encoded'])

# Set sentence length

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

t=np.array(labels)
t=t.astype(float)

max_length = 150
padded_sequences = pad_sequences(sentence_df['Sentences_encoded'], maxlen=max_length, padding='post', truncating='post', value=0.0)

# Convert to NumPy array
x = np.array(padded_sequences)

# Print the results
print(x)

print(t[0:2])
print(x[0:2])



# Prepare Input Vectors and Targets for Training.

In [ ]:
iHateHitler=x[0]
youLikeFish=x[99]
print(iHateHitler)
print(youLikeFish)

print(x[0:5])
print(t[0:5])

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

xx, tt = shuffle(x, t)

# Split the training and testing sets.

In [15]:
train_data, test_data, train_labels, test_labels = train_test_split(xx, tt, test_size=0.2)


# The model that does the word embedding while performing the classification.

In [16]:
embeddingDim=50

model = Sequential([
  Embedding(vocabSize, embeddingDim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(80, activation='relu'),
  Dense(1)
])

In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
model.fit(train_data, train_labels,  epochs=40)


Epoch 1/40
75/75 [==============================] - 15s 114ms/step - loss: 0.6940 - accuracy: 0.4975
Epoch 2/40
75/75 [==============================] - 8s 105ms/step - loss: 0.6930 - accuracy: 0.4975
Epoch 3/40
75/75 [==============================] - 3s 34ms/step - loss: 0.6909 - accuracy: 0.4975
Epoch 4/40
75/75 [==============================] - 2s 24ms/step - loss: 0.6788 - accuracy: 0.4975
Epoch 5/40
75/75 [==============================] - 1s 8ms/step - loss: 0.6362 - accuracy: 0.5508
Epoch 6/40
75/75 [==============================] - 1s 8ms/step - loss: 0.5366 - accuracy: 0.6746
Epoch 7/40
75/75 [==============================] - 1s 8ms/step - loss: 0.4168 - accuracy: 0.8217
Epoch 8/40
75/75 [==============================] - 1s 11ms/step - loss: 0.3129 - accuracy: 0.8788
Epoch 9/40
75/75 [==============================] - 1s 8ms/step - loss: 0.2419 - accuracy: 0.9221
Epoch 10/40
75/75 [==============================] - 1s 13ms/step - loss: 0.1875 - accuracy: 0.9496
Epoch 11/4

In [20]:
test_loss, test_acc = model.evaluate(test_data, test_labels)

print('Test Accuracy:', test_acc, '\n')

19/19 [==============================] - 0s 3ms/step - loss: 0.7368 - accuracy: 0.7700
Test Accuracy: 0.7699999809265137 



# Get the embeddings at the end of training by peeking at the weights of the first layer of the network.

In [21]:
embeddings = model.layers[0].get_weights()[0]
print(embeddings.shape)
a=embeddings[sorteduW.index('hate')]
b=embeddings[sorteduW.index('love')]
print(a)
print('\n')
print(b)
print('\n')
print(np.corrcoef(a,b))

(8018, 50)
[-0.45511377 -0.4727987   0.489451    0.51529014 -0.4871941   0.5250602
  0.56976414  0.5223474  -0.44540945  0.54211515 -0.5266196  -0.4995981
  0.43849072 -0.5359829   0.5497739  -0.47747687 -0.5621909  -0.4837366
 -0.57740307 -0.5103555  -0.5437101   0.5300572   0.50645894  0.43378893
 -0.53797936  0.52490944 -0.48750973 -0.45484254 -0.5568088  -0.48380426
 -0.47505844  0.46042138 -0.44687152  0.4837663  -0.51417077 -0.57063335
  0.4160606   0.50075275 -0.5142611  -0.49497318  0.5437853   0.47397017
 -0.5636819   0.45688453  0.5566985  -0.5097866   0.53517455  0.5458703
 -0.4886364  -0.55113935]


[ 0.61175257  0.6410861  -0.5467578  -0.54610264  0.6634037  -0.6505082
 -0.6973036  -0.6169872   0.57087225 -0.542883    0.6575816   0.5732217
 -0.558132    0.63422984 -0.6775483   0.55701834  0.6219695   0.64351565
  0.6984722   0.5613606   0.66142154 -0.64201385 -0.5535319  -0.5427602
  0.6433262  -0.6125068   0.5767542   0.5835902   0.6869195   0.646871
  0.57499665 -0.54265

# Check out the Coorelation Coefficients!

In [23]:
a=embeddings[sorteduW.index('hate')]   # hate
b=embeddings[sorteduW.index('love')]   # kitten
print(np.corrcoef(a,b))

[[ 1.         -0.99729639]
 [-0.99729639  1.        ]]
